# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 

As my image captioning system is constucted of CNN and RNN, CNN is made of ResNet101 mainly with predetemined weights.

According to [this paper](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf ), ResNet101 has good accuracy compared to ResNet50, but the accuracy of ResNet152 seems to be saturated. That's why I chose ResNet101.
Output from ResNet101 is consumed by linear layer, and linear layer output the feature data.

RNN is made of embedding, LSTM, and linear layer. Embedding layer absorbs captions to generate embedding data. LSTM consumes features extracted by CNN and embedding data. As manner of usage LSTM, I catenate the feature and the embedding data.

To decide each parameters, I referred to [this brilliant paper](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Vinyals_Show_and_Tell_2015_CVPR_paper.pdf) because proposition in this paper is very simple and more powerful among various models proposed over the world.
I set `hidden_size` as 512, `vocab_threshold` as 5 because this google paper explained so.
I chose 512 for `embed_size` because I want `hidden_size` to be equal to `embed_size`.

More layer gave better accuracy in [this paper](https://arxiv.org/pdf/1805.09137) also.
I chose 16 for `batch size`.



### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 16          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    # transforms.RandomRotation(10),                   # get rotation image with random degree
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers=2)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())


# TODO #4: Define the optimizer.
optimizer = optim.Adam(params, lr = 0.0001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

# print model
print(encoder)
print(decoder)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.68s)
creating index...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 667/414113 [00:00<01:02, 6609.54it/s]



index created!
Obtaining caption lengths...


  0%|          | 1478/414113 [00:00<00:56, 7358.88it/s]

  1%|          | 2279/414113 [00:00<00:54, 7573.25it/s]

  1%|          | 3056/414113 [00:00<00:53, 7622.49it/s]

  1%|          | 3840/414113 [00:00<00:53, 7664.35it/s]

  1%|          | 4648/414113 [00:00<00:52, 7733.84it/s]

  1%|▏         | 5483/414113 [00:00<00:52, 7818.60it/s]

  2%|▏         | 6281/414113 [00:00<00:52, 7838.33it/s]

  2%|▏         | 7079/414113 [00:00<00:51, 7853.50it/s]

  2%|▏         | 7877/414113 [00:01<00:51, 7866.11it/s]

  2%|▏         | 8667/414113 [00:01<00:51, 7868.36it/s]

  2%|▏         | 9457/414113 [00:01<00:51, 7871.06it/s]

  2%|▏         | 10274/414113 [00:01<00:51, 7892.25it/s]

  3%|▎         | 11067/414113 [00:01<00:51, 7888.04it/s]

  3%|▎         | 11857/414113 [00:01<00:51, 7880.47it/s]

  3%|▎         | 12645/414113 [00:01<00:51, 7857.51it/s]

  3%|▎         | 13422/414113 [00:01<00:51, 7846.31it/s]

  3%|▎         | 14196/414113 [00:01<00:51, 7834.37it/s]

  4%|▎         | 14973/41

 26%|██▌       | 106512/414113 [00:13<00:38, 8022.75it/s]

 26%|██▌       | 107329/414113 [00:13<00:38, 8023.75it/s]

 26%|██▌       | 108137/414113 [00:13<00:38, 8022.65it/s]

 26%|██▋       | 108981/414113 [00:13<00:38, 8025.75it/s]

 27%|██▋       | 109830/414113 [00:13<00:37, 8029.07it/s]

 27%|██▋       | 110657/414113 [00:13<00:37, 8030.79it/s]

 27%|██▋       | 111483/414113 [00:13<00:37, 8024.83it/s]

 27%|██▋       | 112280/414113 [00:13<00:37, 8024.14it/s]

 27%|██▋       | 113157/414113 [00:14<00:37, 8029.42it/s]

 28%|██▊       | 114054/414113 [00:14<00:37, 8036.00it/s]

 28%|██▊       | 114921/414113 [00:14<00:37, 8040.41it/s]

 28%|██▊       | 115788/414113 [00:14<00:37, 8044.74it/s]

 28%|██▊       | 116653/414113 [00:14<00:36, 8048.86it/s]

 28%|██▊       | 117511/414113 [00:14<00:36, 8051.76it/s]

 29%|██▊       | 118375/414113 [00:14<00:36, 8055.71it/s]

 29%|██▉       | 119253/414113 [00:14<00:36, 8060.59it/s]

 29%|██▉       | 120117/414113 [00:14<00:36, 8064.16it/s

 51%|█████▏    | 212937/414113 [00:26<00:24, 8095.62it/s]

 52%|█████▏    | 213799/414113 [00:26<00:24, 8097.21it/s]

 52%|█████▏    | 214685/414113 [00:26<00:24, 8100.05it/s]

 52%|█████▏    | 215552/414113 [00:26<00:24, 8097.24it/s]

 52%|█████▏    | 216418/414113 [00:26<00:24, 8099.33it/s]

 52%|█████▏    | 217288/414113 [00:26<00:24, 8101.55it/s]

 53%|█████▎    | 218138/414113 [00:26<00:24, 8101.94it/s]

 53%|█████▎    | 218980/414113 [00:27<00:24, 8102.96it/s]

 53%|█████▎    | 219828/414113 [00:27<00:23, 8104.32it/s]

 53%|█████▎    | 220671/414113 [00:27<00:23, 8103.29it/s]

 53%|█████▎    | 221505/414113 [00:27<00:23, 8104.14it/s]

 54%|█████▎    | 222351/414113 [00:27<00:23, 8105.38it/s]

 54%|█████▍    | 223188/414113 [00:27<00:23, 8106.36it/s]

 54%|█████▍    | 224034/414113 [00:27<00:23, 8107.66it/s]

 54%|█████▍    | 224873/414113 [00:27<00:23, 8108.71it/s]

 55%|█████▍    | 225711/414113 [00:27<00:23, 8108.25it/s]

 55%|█████▍    | 226566/414113 [00:27<00:23, 8109.79it/s

 77%|███████▋  | 319994/414113 [00:39<00:11, 8125.07it/s]

 77%|███████▋  | 320891/414113 [00:39<00:11, 8127.18it/s]

 78%|███████▊  | 321798/414113 [00:39<00:11, 8129.55it/s]

 78%|███████▊  | 322687/414113 [00:39<00:11, 8131.22it/s]

 78%|███████▊  | 323574/414113 [00:39<00:11, 8132.17it/s]

 78%|███████▊  | 324449/414113 [00:39<00:11, 8133.33it/s]

 79%|███████▊  | 325320/414113 [00:39<00:10, 8133.87it/s]

 79%|███████▉  | 326180/414113 [00:40<00:10, 8135.00it/s]

 79%|███████▉  | 327039/414113 [00:40<00:10, 8135.39it/s]

 79%|███████▉  | 327889/414113 [00:40<00:10, 8135.34it/s]

 79%|███████▉  | 328741/414113 [00:40<00:10, 8136.27it/s]

 80%|███████▉  | 329591/414113 [00:40<00:10, 8137.13it/s]

 80%|███████▉  | 330436/414113 [00:40<00:10, 8137.22it/s]

 80%|███████▉  | 331273/414113 [00:40<00:10, 8137.65it/s]

 80%|████████  | 332119/414113 [00:40<00:10, 8138.41it/s]

 80%|████████  | 332988/414113 [00:40<00:09, 8139.74it/s]

 81%|████████  | 333842/414113 [00:41<00:09, 8140.65it/s

EncoderCNN(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d(64, 256, ker

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [15]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
# response = requests.request("GET", 
#                             "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            # requests.request("POST", 
            #                  "https://nebula.udacity.com/api/v1/remote/keep-alive", 
            #                  headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/25883], Loss: 5.2812, Perplexity: 196.6101
Epoch [1/3], Step [200/25883], Loss: 4.8481, Perplexity: 127.4955
Epoch [1/3], Step [300/25883], Loss: 4.6740, Perplexity: 107.1265
Epoch [1/3], Step [400/25883], Loss: 4.5426, Perplexity: 93.93216
Epoch [1/3], Step [500/25883], Loss: 4.4909, Perplexity: 89.19793
Epoch [1/3], Step [600/25883], Loss: 4.5164, Perplexity: 91.50289
Epoch [1/3], Step [700/25883], Loss: 4.0797, Perplexity: 59.12980
Epoch [1/3], Step [800/25883], Loss: 4.2317, Perplexity: 68.83623
Epoch [1/3], Step [900/25883], Loss: 4.1595, Perplexity: 64.03875
Epoch [1/3], Step [1000/25883], Loss: 4.3620, Perplexity: 78.4174
Epoch [1/3], Step [1100/25883], Loss: 4.2236, Perplexity: 68.27603
Epoch [1/3], Step [1200/25883], Loss: 4.3322, Perplexity: 76.11134
Epoch [1/3], Step [1300/25883], Loss: 4.0815, Perplexity: 59.23150
Epoch [1/3], Step [1400/25883], Loss: 4.0445, Perplexity: 57.08368
Epoch [1/3], Step [1500/25883], Loss: 3.6502, Perplexity: 38.48156
Epoch

Epoch [1/3], Step [12400/25883], Loss: 3.0889, Perplexity: 21.9538
Epoch [1/3], Step [12500/25883], Loss: 3.1876, Perplexity: 24.2308
Epoch [1/3], Step [12600/25883], Loss: 2.2407, Perplexity: 9.40038
Epoch [1/3], Step [12700/25883], Loss: 2.3447, Perplexity: 10.4300
Epoch [1/3], Step [12800/25883], Loss: 2.7288, Perplexity: 15.3139
Epoch [1/3], Step [12900/25883], Loss: 2.6298, Perplexity: 13.87089
Epoch [1/3], Step [13000/25883], Loss: 2.6733, Perplexity: 14.4878
Epoch [1/3], Step [13100/25883], Loss: 2.3989, Perplexity: 11.0107
Epoch [1/3], Step [13200/25883], Loss: 2.7169, Perplexity: 15.1333
Epoch [1/3], Step [13300/25883], Loss: 2.7072, Perplexity: 14.9867
Epoch [1/3], Step [13400/25883], Loss: 2.7186, Perplexity: 15.1586
Epoch [1/3], Step [13500/25883], Loss: 2.4958, Perplexity: 12.1312
Epoch [1/3], Step [13600/25883], Loss: 2.7903, Perplexity: 16.2857
Epoch [1/3], Step [13700/25883], Loss: 2.9079, Perplexity: 18.3179
Epoch [1/3], Step [13800/25883], Loss: 2.6367, Perplexity: 13

Epoch [1/3], Step [24600/25883], Loss: 2.4256, Perplexity: 11.3085
Epoch [1/3], Step [24700/25883], Loss: 2.4829, Perplexity: 11.9754
Epoch [1/3], Step [24800/25883], Loss: 2.7146, Perplexity: 15.0988
Epoch [1/3], Step [24900/25883], Loss: 2.3137, Perplexity: 10.1118
Epoch [1/3], Step [25000/25883], Loss: 2.3281, Perplexity: 10.2582
Epoch [1/3], Step [25100/25883], Loss: 2.5263, Perplexity: 12.5069
Epoch [1/3], Step [25200/25883], Loss: 2.2622, Perplexity: 9.60423
Epoch [1/3], Step [25300/25883], Loss: 2.1959, Perplexity: 8.98853
Epoch [1/3], Step [25400/25883], Loss: 2.8765, Perplexity: 17.7513
Epoch [1/3], Step [25500/25883], Loss: 2.3938, Perplexity: 10.9554
Epoch [1/3], Step [25600/25883], Loss: 2.8652, Perplexity: 17.5522
Epoch [1/3], Step [25700/25883], Loss: 2.4292, Perplexity: 11.3496
Epoch [1/3], Step [25800/25883], Loss: 2.6633, Perplexity: 14.3439
Epoch [2/3], Step [100/25883], Loss: 2.1611, Perplexity: 8.6811185
Epoch [2/3], Step [200/25883], Loss: 2.1801, Perplexity: 8.846

Epoch [2/3], Step [11100/25883], Loss: 2.8221, Perplexity: 16.8117
Epoch [2/3], Step [11200/25883], Loss: 2.4012, Perplexity: 11.0366
Epoch [2/3], Step [11300/25883], Loss: 2.2784, Perplexity: 9.761027
Epoch [2/3], Step [11400/25883], Loss: 2.4563, Perplexity: 11.6620
Epoch [2/3], Step [11500/25883], Loss: 2.7774, Perplexity: 16.0767
Epoch [2/3], Step [11600/25883], Loss: 2.6257, Perplexity: 13.8136
Epoch [2/3], Step [11700/25883], Loss: 2.0088, Perplexity: 7.45421
Epoch [2/3], Step [11800/25883], Loss: 2.5963, Perplexity: 13.4142
Epoch [2/3], Step [11900/25883], Loss: 2.8233, Perplexity: 16.8317
Epoch [2/3], Step [12000/25883], Loss: 2.2293, Perplexity: 9.29336
Epoch [2/3], Step [12100/25883], Loss: 2.6193, Perplexity: 13.7262
Epoch [2/3], Step [12200/25883], Loss: 2.0870, Perplexity: 8.06061
Epoch [2/3], Step [12300/25883], Loss: 2.5591, Perplexity: 12.9237
Epoch [2/3], Step [12400/25883], Loss: 2.3909, Perplexity: 10.9230
Epoch [2/3], Step [12500/25883], Loss: 2.1495, Perplexity: 8.

Epoch [2/3], Step [23300/25883], Loss: 2.0628, Perplexity: 7.86809
Epoch [2/3], Step [23400/25883], Loss: 2.4687, Perplexity: 11.8073
Epoch [2/3], Step [23500/25883], Loss: 2.9717, Perplexity: 19.5259
Epoch [2/3], Step [23600/25883], Loss: 2.6038, Perplexity: 13.5155
Epoch [2/3], Step [23700/25883], Loss: 2.6017, Perplexity: 13.4864
Epoch [2/3], Step [23800/25883], Loss: 2.1788, Perplexity: 8.83590
Epoch [2/3], Step [23900/25883], Loss: 2.1041, Perplexity: 8.19946
Epoch [2/3], Step [24000/25883], Loss: 2.2413, Perplexity: 9.40553
Epoch [2/3], Step [24100/25883], Loss: 2.2811, Perplexity: 9.78791
Epoch [2/3], Step [24200/25883], Loss: 2.9244, Perplexity: 18.6230
Epoch [2/3], Step [24300/25883], Loss: 2.6248, Perplexity: 13.8012
Epoch [2/3], Step [24400/25883], Loss: 2.1224, Perplexity: 8.350991
Epoch [2/3], Step [24500/25883], Loss: 2.7125, Perplexity: 15.0676
Epoch [2/3], Step [24600/25883], Loss: 2.3842, Perplexity: 10.8500
Epoch [2/3], Step [24700/25883], Loss: 2.3223, Perplexity: 10

Epoch [3/3], Step [9800/25883], Loss: 1.9477, Perplexity: 7.01243
Epoch [3/3], Step [9900/25883], Loss: 1.8409, Perplexity: 6.30221
Epoch [3/3], Step [10000/25883], Loss: 2.4671, Perplexity: 11.7887
Epoch [3/3], Step [10100/25883], Loss: 2.5906, Perplexity: 13.3375
Epoch [3/3], Step [10200/25883], Loss: 2.0005, Perplexity: 7.39288
Epoch [3/3], Step [10300/25883], Loss: 2.1788, Perplexity: 8.83603
Epoch [3/3], Step [10400/25883], Loss: 2.3509, Perplexity: 10.4953
Epoch [3/3], Step [10500/25883], Loss: 2.5131, Perplexity: 12.3431
Epoch [3/3], Step [10600/25883], Loss: 2.1692, Perplexity: 8.75156
Epoch [3/3], Step [10700/25883], Loss: 2.4086, Perplexity: 11.1181
Epoch [3/3], Step [10800/25883], Loss: 2.4021, Perplexity: 11.0463
Epoch [3/3], Step [10900/25883], Loss: 2.8693, Perplexity: 17.6244
Epoch [3/3], Step [11000/25883], Loss: 1.8849, Perplexity: 6.58548
Epoch [3/3], Step [11100/25883], Loss: 2.5123, Perplexity: 12.3328
Epoch [3/3], Step [11200/25883], Loss: 2.4576, Perplexity: 11.67

Epoch [3/3], Step [22000/25883], Loss: 2.5977, Perplexity: 13.4327
Epoch [3/3], Step [22100/25883], Loss: 2.6270, Perplexity: 13.8324
Epoch [3/3], Step [22200/25883], Loss: 2.0841, Perplexity: 8.03735
Epoch [3/3], Step [22300/25883], Loss: 2.8242, Perplexity: 16.8478
Epoch [3/3], Step [22400/25883], Loss: 2.2383, Perplexity: 9.37778
Epoch [3/3], Step [22500/25883], Loss: 2.6696, Perplexity: 14.4345
Epoch [3/3], Step [22600/25883], Loss: 2.1663, Perplexity: 8.72594
Epoch [3/3], Step [22700/25883], Loss: 2.5344, Perplexity: 12.6092
Epoch [3/3], Step [22800/25883], Loss: 2.3104, Perplexity: 10.0787
Epoch [3/3], Step [22900/25883], Loss: 2.1554, Perplexity: 8.63165
Epoch [3/3], Step [23000/25883], Loss: 2.0528, Perplexity: 7.78990
Epoch [3/3], Step [23100/25883], Loss: 2.6730, Perplexity: 14.4831
Epoch [3/3], Step [23200/25883], Loss: 2.3553, Perplexity: 10.5411
Epoch [3/3], Step [23300/25883], Loss: 2.1088, Perplexity: 8.23845
Epoch [3/3], Step [23400/25883], Loss: 2.0172, Perplexity: 7.5

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.